In [1]:
import os

import numpy as np
import flopy

In [2]:
def rmse(sim, obs, locs):
    acc = 0.
    for loc in locs:
        diff = sim[loc] - obs[loc]
        acc += diff ** 2.
    return np.sqrt(acc / np.float(len(locs)))

In [3]:
# load the base model
bws = os.path.join('..', 'base')
m = flopy.modflow.Modflow.load('m.nam', version='mf2005', model_ws=bws, check=False)

In [4]:
# observation location
obs_locs = [(0,3,10),
            (0,3,16),
            (0,4,9),
            (0,10,2),
            (0,14,11),
            (0,16,17),
            (0,22,11),
            (0,23,16),
            (0,25,5),
            (0,27,7),
            (0,30,16),
            (0,34,8),
            (0,35,11),
            (0,39,8),
            (0,39,11),
            (0,39,13)]

In [5]:
# get list of active cells
active_locs = []
ib = m.bas6.ibound.array
for i in range(ib.shape[1]):
    for j in range(ib.shape[2]):
        if ib[0, i, j] > 0:
            active_locs.append((0, i, j))

In [6]:
# get list of pumping well locations
wels = m.wel.stress_period_data[1]
well_locs = []
for wel in wels:
    well_locs.append((wel['k'], wel['i'], wel['j']))

In [7]:
# get list of hk files
dpth = os.path.join('..', 'hk')
hk_files = [os.path.join(dpth, fn) for fn in sorted(os.listdir(dpth)) 
            if os.path.splitext(os.path.join(dpth, fn))[1] in ['.DAT', '.dat']]
hk_files

['../hk/GroupA_HK.DAT',
 '../hk/GroupB_HK.DAT',
 '../hk/GroupC_HK.dat',
 '../hk/GroupC_prediction_HK.dat',
 '../hk/GroupD_HK.dat',
 '../hk/GroupE_EX04_HK.DAT',
 '../hk/GroupF_HK.DAT',
 '../hk/GroupG_freyberg_KH_matrix.dat']

In [8]:
# get list of calibration head files
dpth = os.path.join('..', 'calibration-heads')
calh_files = [os.path.join(dpth, fn) for fn in sorted(os.listdir(dpth)) if '.hds' in fn]

In [9]:
# get list of predictive head files
dpth = os.path.join('..', 'predictive-heads')
predh_files = [os.path.join(dpth, fn) for fn in sorted(os.listdir(dpth)) if '.hds' in fn]

In [10]:
# get true hk
hktrue = m.lpf.hk.array
hktrue.shape
shape2d = hktrue[0, :, :].shape
shape3d = hktrue.shape

In [11]:
# get true calibration and predictive heads
fpth = os.path.join(bws, 'm.hds')
hobj = flopy.utils.HeadFile(fpth)
ch = hobj.get_data(totim=1.) 
ph = hobj.get_data(totim=2.)

In [12]:
print('Observation Wells - Hydraulic Conductivity')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in hk_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    d = np.loadtxt(fpth)
    if d.shape != shape2d:
        d = flopy.utils.Util2d.load_txt(shape2d, fpth, fmtin='(10e15.6)', dtype=np.float32)
    d = d.reshape(shape3d)
    v = rmse(d, hktrue, obs_locs)
    print('{:5s}{:20g}'.format(sid, v))

Observation Wells - Hydraulic Conductivity
Group                RMSE
-------------------------
A                 2.53568
B                 5.45795
C                  3.0153
C                  3.0153
D                 2.43022
E                 2.22261
F                 2.90602
G                 1.39776


In [13]:
print('Active Cells - Hydraulic Conductivity')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in hk_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    d = np.loadtxt(fpth)
    if d.shape != shape2d:
        d = flopy.utils.Util2d.load_txt(shape2d, fpth, fmtin='(10e15.6)', dtype=np.float32)
    d = d.reshape(shape3d)
    v = rmse(d, hktrue, active_locs)
    print('{:5s}{:20g}'.format(sid, v))

Active Cells - Hydraulic Conductivity
Group                RMSE
-------------------------
A                 2.25461
B                 4.09774
C                 2.30845
C                 2.30845
D                 2.35268
E                 1.83573
F                 2.24387
G                 1.96397


In [14]:
print('Observation Well - Calibration')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in calh_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    hobj = flopy.utils.HeadFile(fpth)
    h = hobj.get_data()
    v = rmse(h, ch, obs_locs)
    print('{:5s}{:20g}'.format(sid, v))
    #rmseobs.append(rmse(h, calhtrue, active_locs))
    #rmsecalh.append(rmse(h, calhtrue, obs_locs))
    

Observation Well - Calibration
Group                RMSE
-------------------------
A                 1.28112
C                0.538722
E                 1.27793
G               0.0752089


In [15]:
print('Active Cells - Calibration')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in calh_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    hobj = flopy.utils.HeadFile(fpth)
    h = hobj.get_data()
    v = rmse(h, ch, active_locs)
    print('{:5s}{:20g}'.format(sid, v))

Active Cells - Calibration
Group                RMSE
-------------------------
A                0.815865
C                0.604773
E                 1.24863
G                0.262018


In [16]:
print('Observation Well - Prediction')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in predh_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    hobj = flopy.utils.HeadFile(fpth)
    h = hobj.get_data()
    v = rmse(h, ph, obs_locs)
    print('{:5s}{:20g}'.format(sid, v))

Observation Well - Prediction
Group                RMSE
-------------------------
A                 1.05826
B                0.961334
C                0.535769
D                0.597594
E                0.110573
F                0.562801
G               0.0852771


In [17]:
print('Pumping Well - Prediction')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in predh_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    hobj = flopy.utils.HeadFile(fpth)
    h = hobj.get_data()
    v = rmse(h, ph, well_locs)
    print('{:5s}{:20g}'.format(sid, v))

Pumping Well - Prediction
Group                RMSE
-------------------------
A                 1.69158
B                 1.73671
C                 2.08779
D                   1.892
E                 1.01928
F             4.08248e+29
G                 1.06388


In [18]:
print('Active Cells - Prediction')
print('{:5s}{:>20s}'.format('Group', 'RMSE'))
print(25*('-'))
for fpth in predh_files:
    sid = os.path.basename(fpth).split('_')[0].replace('Group', '')
    hobj = flopy.utils.HeadFile(fpth)
    h = hobj.get_data()
    v = rmse(h, ph, active_locs)
    print('{:5s}{:20g}'.format(sid, v))

Active Cells - Prediction
Group                RMSE
-------------------------
A                0.796085
B                0.966877
C                0.626479
D                0.596279
E                0.246694
F             3.76622e+28
G                0.273047
